In [ ]:
import sys, os, copy, time
import random
import numpy as np
import torch
HOME_PATH = os.path.expanduser('~')

import torch.nn as nn
import torch.optim as optim
from tensorboardX import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import poisevae
from poisevae.datasets import MNIST_MNIST
from poisevae.networks.PixelCNN_MNIST import EncMNIST, DecMNIST
from poisevae.networks.pixelcnn import PixelCNN

In [ ]:
PATH = os.path.join(HOME_PATH, 'vae_project/POISEVAE/examples')

In [ ]:
MNIST_PATH = os.path.join(HOME_PATH, 'Datasets/MNIST/%s.pt')

joint_dataset_train = MNIST_MNIST(mnist_pt_path=MNIST_PATH % 'train')
joint_dataset_test = MNIST_MNIST(mnist_pt_path=MNIST_PATH % 'test')

In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(joint_dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(joint_dataset_test, batch_size=batch_size, shuffle=True, drop_last=True)
len(train_loader), len(test_loader)

In [ ]:
lat1, lat2 = 20, 20
color_level = 256
device = 'cuda' if torch.cuda.is_available() else 'cpu'

enc_mnist1 = EncMNIST(lat1).to(device)
dec_mnist1 = DecMNIST(PixelCNN(lat1, 1, color_level), color_level).to(device)
enc_mnist2 = EncMNIST(lat1).to(device)
dec_mnist2 = DecMNIST(PixelCNN(lat2, 1, color_level), color_level).to(device)

# Options: 'derivative_autograd', 'derivative_gradient', and 'std_normal'
vae = poisevae.POISEVAE([enc_mnist1, enc_mnist2], [dec_mnist1, dec_mnist2], latent_dims=[lat1, (lat2)], 
                        enc_config='nu', KL_calc='derivative_autograd', batch_size=batch_size
                       ).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
# writer = None
writer = SummaryWriter(PATH)

In [ ]:
epoch = 170

In [ ]:
vae, _, epoch = poisevae.utils.load_checkpoint(vae, load_path=os.path.join(PATH, 'training_170.pt'))

In [ ]:
epochs = 50 + epoch
for epoch in tqdm(range(epoch, epochs)):
    poisevae.utils.train(vae, train_loader, optimizer, epoch, kl_weight=1, n_gibbs_iter=5, writer=writer, 
                         dec_kwargs={'generate_mode':False})
    poisevae.utils.test(vae, test_loader, epoch, kl_weight=1, n_gibbs_iter=5, writer=writer, 
                        dec_kwargs={'generate_mode':False})
    if (epoch+1) % 10 == 0 and epoch > 0:
        poisevae.utils.save_checkpoint(vae, optimizer, os.path.join(PATH, 'training_%d.pt' % (epoch+1)), epoch+1) 

In [ ]:
for epoch in (170, 180, 190, 200, 210, 220):
    vae, _, epoch = poisevae.utils.load_checkpoint(vae, load_path=os.path.join(PATH, 'training_%d.pt') % epoch)
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            results = vae([data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.float32)], 
                          dec_kwargs={'generate_mode':False})
            print(results['total_loss'].item(), results['KL_loss'], 
                  results['rec_losses'][0], results['rec_losses'][1], sep='\t\t')
            break

## Reconstruction

In [ ]:
with torch.no_grad():
    for i, data in enumerate(test_loader):
        results = vae([data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.float32)], 
                      dec_kwargs={'generate_mode':False})
        break

results['x_rec'][0].shape

In [ ]:
ncols = min(len(results['x_rec'][0]), 20)
fig, ax = plt.subplots(nrows=2, ncols=ncols, figsize=(15, 1.5))
for i, aux in enumerate(zip(data[0], results['x_rec'][0].max(dim=1).indices)):
    # for i, aux in enumerate(zip(data[0], results['x_rec'][0][:, 230])):
    if i >= ncols:
        break
    for j, im in enumerate(aux):
        ax[j, i].imshow(im.cpu().numpy().reshape(28, 28), cmap='gray')
        ax[j, i].set_axis_off()
# ax[1, 0].set_ylabel('Rec', fontsize=24)
fig.tight_layout(pad=0)
# fig.savefig(PATH + 'MNISTRec.pdf', dpi=300)

## Cross Generation

In [ ]:
with torch.no_grad():
    for i, data in enumerate(test_loader):
        results = vae([data[0].to(device, dtype=torch.float32), None], 
                      dec_kwargs={'generate_mode':True})
        break

results['x_rec'][0].shape

In [ ]:
ncols = min(len(results['x_rec'][0]), 20)
fig, ax = plt.subplots(nrows=2, ncols=ncols, figsize=(15, 1.5))
for i, aux in enumerate(zip(data[0], results['x_rec'][1])):
# for i, aux in enumerate(zip(data[0], results['x_rec'][1].max(dim=1).indices)):
    if i >= ncols:
        break
    for j, im in enumerate(aux):
        ax[j, i].imshow(im.cpu().numpy().reshape(28, 28), cmap='gray')
        ax[j, i].set_axis_off()
# ax[1, 0].set_ylabel('Rec', fontsize=24)
fig.tight_layout(pad=0)
# fig.savefig(PATH + 'MNISTRec.pdf', dpi=300)

In [ ]:
with torch.no_grad():
    for i, data in enumerate(test_loader):
        results = vae([None, data[1].to(device, dtype=torch.float32)], 
                      dec_kwargs={'generate_mode':True})
        break

results['x_rec'][0].shape

In [ ]:
ncols = min(len(results['x_rec'][0]), 20)
fig, ax = plt.subplots(nrows=2, ncols=ncols, figsize=(15, 1.5))
for i, aux in enumerate(zip(data[1], results['x_rec'][0])):
# for i, aux in enumerate(zip(data[1], results['x_rec'][0].max(dim=1).indices)):
    if i >= ncols:
        break
    for j, im in enumerate(aux):
        ax[j, i].imshow(im.cpu().numpy().reshape(28, 28), cmap='gray')
        ax[j, i].set_axis_off()
# ax[1, 0].set_ylabel('Rec', fontsize=24)
fig.tight_layout(pad=0)
# fig.savefig(PATH + 'MNISTRec.pdf', dpi=300)

## Joint Generation (conditioned)

In [ ]:
with torch.no_grad():
    for i, data in enumerate(test_loader):
        results = vae([data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.float32)], 
                      dec_kwargs={'generate_mode':True})
        break

results['x_rec'][0].shape

In [ ]:
ncols = min(len(results['x_rec'][0]), 20)
fig, ax = plt.subplots(nrows=2, ncols=ncols, figsize=(15, 1.5))
for i, aux in enumerate(zip(results['x_rec'][0], results['x_rec'][1])):
    if i >= ncols:
        break
    for j, im in enumerate(aux):
        ax[j, i].imshow(im.cpu().numpy().reshape(28, 28), cmap='gray')
        ax[j, i].set_axis_off()
# ax[1, 0].set_ylabel('Rec', fontsize=24)
fig.tight_layout(pad=0)
# fig.savefig(PATH + 'MNISTRec.pdf', dpi=300)

## Joint Generation (not conditioned)

In [ ]:
with torch.no_grad():
    results = vae.generate(n_samples=12, dec_kwargs={'generate_mode':True})
results['x_rec'][0].shape

In [ ]:
ncols = min(len(results['x_rec'][0]), 20)
fig, ax = plt.subplots(nrows=2, ncols=ncols, figsize=(15, 1.5))
for i, aux in enumerate(zip(results['x_rec'][0], results['x_rec'][1])):
    if i >= ncols:
        break
    for j, im in enumerate(aux):
        ax[j, i].imshow(im.cpu().numpy().reshape(28, 28), cmap='gray')
        ax[j, i].set_axis_off()
# ax[1, 0].set_ylabel('Rec', fontsize=24)
fig.tight_layout(pad=0)
# fig.savefig(PATH + 'MNISTRec.pdf', dpi=300)